In [2]:
import pandas as pd
import numpy as np

# Introduction/Business Problem

A business owner is looking to open a Tex-Mex restaurant in Seattle or one of its accompanying suburbs. Which area of the city is best to open a their restaurant?

# Data Needed

1. List of neighborhoods and their coordinates near Seattle
2. Geocoordinates of neighborhoods near seattle
3. Restaurants located within each neighborhood